In [ ]:
from tqdm import tqdm
import pandas as pd
import ast
import pickle

# reading the files 
with open("main_bar_similarity_list", "rb") as fp1: 
  visible_similarity_matrix_list = pickle.load(fp1) 

with open("sidebar_similarity_list", "rb") as fp2: 
  non_visible_similarity_matrix_list = pickle.load(fp2) 

with open("recommendation_id", "rb") as fp3: 
  recommendation_id_list = pickle.load(fp3)

# expanding the dataset vertically so we can perform our analysis in R
def extending_out(id_list,original_list_of_list):
  new_list,new_id_list,order,=[],[],[]
  for i in tqdm(range(len(original_list_of_list))):
    current=original_list_of_list[i]
    id=str(id_list[i])
    for j in range(len(current)):
      t=int(j+1)
      score=float(current[j])
      new_list.append(score)
      new_id_list.append(id)
      order.append(t)
  return(order,new_id_list,new_list)

# executing the verticle expansion for the default observed comments 
order,new_recommendation_ids,observable_comments_list=extending_out(recommendation_id_list,visible_similarity_matrix_list)
typeof=["mainbar" for i in range(len(new_recommendation_ids))]
df1 = pd.DataFrame(list(zip(new_recommendation_ids,observable_comments_list,order,typeof)), columns=['recommendation_id','similarity_score','order','type'])  

# executing the verticle expansion for the side-bar display of recent reviews
order,new_recommendation_ids,unobservable_comments_list=extending_out(recommendation_id_list,non_visible_similarity_matrix_list)
typeof=["sidebar" for i in range(len(new_recommendation_ids))]
df2 = pd.DataFrame(list(zip(new_recommendation_ids,unobservable_comments_list,order,typeof)), columns =['recommendation_id','similarity_score','order','type'])  

# combining the 2 together in one dataframe 
frames = [df1, df2]
data= pd.concat(frames)
data.to_csv('reviewer_analysis.csv')
data